1. Разложение данных в разные таблички df
2. Обучение разных моделек на своих цветах
3. Соединение результатов

In [21]:
#from google.colab import drive
#drive.mount('/content/drive')

In [22]:
#!pip install opencv-python

In [292]:
import numpy as np
import scipy.linalg as sla
from PIL import Image
from scipy import misc
import scipy.sparse as sps
import scipy.sparse.linalg as spla
import timeit
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix

from matplotlib import cm

from tqdm.auto import tqdm

## Сбор датасетов

In [280]:
name_folder = {0:'1_предупреждающие', 1:'2_приоритета', 2:'3_запрещающие', 3:'4_предписывающие', 4:'5_особых предписаний', 5:'6_информационные', 6:'7_сервиса', 7:'8_доп информации(таблички)'}

In [281]:
# df_0 = pd.DataFrame(columns=[i for i in range(50176)])
# df_1 = pd.DataFrame(columns=[i for i in range(50176)])
# df_2 = pd.DataFrame(columns=[i for i in range(50176)])

# l = 0

# for b in name_folder.keys():

#   path = '/content/drive/MyDrive/Colab Notebooks/НИС Данные/НИС/НИС'+f'/{name_folder[b]}'

#   photos = os.listdir(path)

#   img_sl = {}

#   for i, photo in enumerate(tqdm(photos)):

#     l =+ 1

#     photo_pth = f'{path}/'+f'{photo}'     

#     img = 255 - cv2.imread(photo_pth, cv2.COLOR_BGR2RGB)

#     for n in range(3):

#       data0 = np.reshape(img[:,:,n], 50176)
#       data0 = pd.Series(data0)
#       img_sl[n] = data0

#     df_0 = df_0.append(img_sl[0], ignore_index=True)
#     df_0.loc[(df_0.shape[0] - 1), 'target'] = b

#     df_1 = df_1.append(img_sl[1], ignore_index=True)
#     df_1.loc[(df_0.shape[0] - 1), 'target'] = b

#     df_2 = df_2.append(img_sl[2], ignore_index=True)
#     df_2.loc[(df_2.shape[0] - 1), 'target'] = b

In [282]:
# df_0.to_csv('/content/drive/MyDrive/Colab Notebooks/НИС Данные/НИС/НИС/df_0.csv')
# df_1.to_csv('/content/drive/MyDrive/Colab Notebooks/НИС Данные/НИС/НИС/df_1.csv')
# df_2.to_csv('/content/drive/MyDrive/Colab Notebooks/НИС Данные/НИС/НИС/df_2.csv')


## Загрузим датасеты

In [59]:
df_0 = pd.read_csv('C:/Users/nicol/Python/ВШЭ/НИС/df_0.csv', index_col = 0)
df_1 = pd.read_csv('C:/Users/nicol/Python/ВШЭ/НИС/df_1.csv', index_col = 0)
df_2 = pd.read_csv('C:/Users/nicol/Python/ВШЭ/НИС/df_2.csv', index_col = 0)

In [42]:
df_0.head()

,0,1,2,3,4,5,6,7,8,9,...,50167,50168,50169,50170,50171,50172,50173,50174,50175,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Функция для обучения 

In [234]:
def model_ka(df_0, df_1, df_2, mod_0, mod_1, mod_2):
   
    # разбиение индексов
    x_train_ind, x_test_ind = train_test_split(df_0.index, test_size=0.3, random_state=42)
    
    # разбивка данных для обучения моделек
    x_y_train_0 = df_0.loc[x_train_ind]
    x_y_train_1 = df_1.loc[x_train_ind]
    x_y_train_2 = df_2.loc[x_train_ind]

    x_y_test_0 = df_0.loc[x_test_ind]
    x_y_test_1 = df_1.loc[x_test_ind]
    x_y_test_2 = df_2.loc[x_test_ind]


    x_train_0 = x_y_train_0.drop('target', axis=1)
    x_train_1 = x_y_train_1.drop('target', axis=1)
    x_train_2 = x_y_train_2.drop('target', axis=1)

    x_test_0 = x_y_test_0.drop('target', axis=1)
    x_test_1 = x_y_test_1.drop('target', axis=1)
    x_test_2 = x_y_test_2.drop('target', axis=1)

    y_train = x_y_train_0['target']

    y_test = x_y_test_0['target']
    
    
    # обучение моделек, получение предсказаний для каждой
 
    mod_0.fit(x_train_0, y_train)
    mod_1.fit(x_train_1, y_train)
    mod_2.fit(x_train_2, y_train)
    
    pred_0 = mod_0.predict(x_test_0)
    pred_1 = mod_1.predict(x_test_1)
    pred_2 = mod_2.predict(x_test_2)
    
    
    # самое часто встречающееся значение среди всех предсказаний через медиану
    try:
        pred = list(map(np.median, (list(zip(pred_0[:,0],pred_1[:,0],pred_2[:,0])))))
    
    except IndexError:

        pred = list(map(np.median, (list(zip(pred_0,pred_1,pred_2)))))
    
    # считаем accuracy каждой модельки и суммарное
    
    accuracy_0 = accuracy_score(pred_0, y_test)
    accuracy_1 = accuracy_score(pred_1, y_test)
    accuracy_2 = accuracy_score(pred_2, y_test)
    
    accuracy_vseh = accuracy_score(pred, y_test)
    
    itog = {'1_канал': accuracy_0, '2_канал': accuracy_1, '3_канал':accuracy_2, 'суммарное':accuracy_vseh}
    
    return itog

## Обучим модели

### SGDClassifier

In [238]:
cl_0 = SGDClassifier()
cl_1 = SGDClassifier()
cl_2 = SGDClassifier()

In [239]:
model_ka(df_0, df_1, df_2, cl_0, cl_1, cl_2)

{'1_канал': 0.748792270531401,
 '2_канал': 0.7777777777777778,
 '3_канал': 0.7729468599033816,
 'суммарное': 0.782608695652174}

In [283]:
cl_0 = SGDClassifier(penalty = 'l1')
cl_1 = SGDClassifier(penalty = 'l1')
cl_2 = SGDClassifier(penalty = 'l1')

In [284]:
m_sgd = model_ka(df_0, df_1, df_2, cl_0, cl_1, cl_2)
m_sgd

{'1_канал': 0.7777777777777778,
 '2_канал': 0.8260869565217391,
 '3_канал': 0.782608695652174,
 'суммарное': 0.8260869565217391}

### SVC

In [244]:
svc_0 = SVC(kernel='rbf')
svc_1 = SVC(kernel='rbf')
svc_2 = SVC(kernel='rbf')

In [245]:
model_ka(df_0, df_1, df_2, svc_0, svc_1, svc_2)

{'1_канал': 0.6666666666666666,
 '2_канал': 0.7632850241545893,
 '3_канал': 0.748792270531401,
 'суммарное': 0.7584541062801933}

In [285]:
svc_0 = SVC(kernel='linear')
svc_1 = SVC(kernel='linear')
svc_2 = SVC(kernel='linear')

In [286]:
m_svc = model_ka(df_0, df_1, df_2, svc_0, svc_1, svc_2)
m_svc

{'1_канал': 0.7874396135265701,
 '2_канал': 0.8260869565217391,
 '3_канал': 0.7971014492753623,
 'суммарное': 0.8405797101449275}

### KNeighborsClassifier

In [241]:
cn_0 = KNeighborsClassifier(n_jobs = -1)
cn_1 = KNeighborsClassifier(n_jobs = -1)
cn_2 = KNeighborsClassifier(n_jobs = -1)

In [242]:
model_ka(df_0, df_1, df_2, cn_0, cn_1, cn_2)

{'1_канал': 0.6183574879227053,
 '2_канал': 0.6956521739130435,
 '3_канал': 0.714975845410628,
 'суммарное': 0.6859903381642513}

In [255]:
cn_0 = KNeighborsClassifier(n_jobs = -1, weights='distance')
cn_1 = KNeighborsClassifier(n_jobs = -1, weights='distance')
cn_2 = KNeighborsClassifier(n_jobs = -1, weights='distance')

In [256]:
model_ka(df_0, df_1, df_2, cn_0, cn_1, cn_2)

{'1_канал': 0.6231884057971014,
 '2_канал': 0.7101449275362319,
 '3_канал': 0.748792270531401,
 'суммарное': 0.6956521739130435}

In [263]:
cn_0 = KNeighborsClassifier(n_jobs = -1, weights='distance', n_neighbors = 2)
cn_1 = KNeighborsClassifier(n_jobs = -1, weights='distance', n_neighbors = 2)
cn_2 = KNeighborsClassifier(n_jobs = -1, weights='distance', n_neighbors = 2)

In [287]:
m_kn = model_ka(df_0, df_1, df_2, cn_0, cn_1, cn_2)
m_kn

{'1_канал': 0.714975845410628,
 '2_канал': 0.7777777777777778,
 '3_канал': 0.8357487922705314,
 'суммарное': 0.8019323671497585}

### RandomForestClassifier

In [289]:
rf_0 = RandomForestClassifier()
rf_1 = RandomForestClassifier()
rf_2 = RandomForestClassifier()

In [290]:
m_rf = model_ka(df_0, df_1, df_2, rf_0, rf_1, rf_2)
m_rf

{'1_канал': 0.7729468599033816,
 '2_канал': 0.7729468599033816,
 '3_канал': 0.7777777777777778,
 'суммарное': 0.8019323671497585}

In [278]:
rf_0 = RandomForestClassifier(max_depth = 9)
rf_1 = RandomForestClassifier(max_depth = 9)
rf_2 = RandomForestClassifier(max_depth = 9)

In [279]:
model_ka(df_0, df_1, df_2, rf_0, rf_1, rf_2)

{'1_канал': 0.7632850241545893,
 '2_канал': 0.7971014492753623,
 '3_канал': 0.7632850241545893,
 'суммарное': 0.7922705314009661}

### CatBoost

In [192]:
clf_0 = CatBoostClassifier(loss_function='MultiClass',depth = 3,\
                                                iterations = 180,\
                                                l2_leaf_reg = 1,\
                                                learning_rate = 0.25075,
                                                verbose=False)

In [194]:
clf_1 = CatBoostClassifier(loss_function='MultiClass',depth = 3,\
                                                iterations = 180,\
                                                l2_leaf_reg = 1,\
                                                learning_rate = 0.25075,
                                                verbose=False)

In [196]:
clf_2 = CatBoostClassifier(loss_function='MultiClass',depth = 3,\
                                                iterations = 180,\
                                                l2_leaf_reg = 1,\
                                                learning_rate = 0.25075,
                                                verbose=False)

In [291]:
m_cb = model_ka(df_0, df_1, df_2, clf_0, clf_1, clf_2)
m_cb

{'1_канал': 0.7246376811594203,
 '2_канал': 0.7632850241545893,
 '3_канал': 0.7971014492753623,
 'суммарное': 0.7681159420289855}

## Графики и вывод

In [303]:
list(m_sgd.keys())

['1_канал', '2_канал', '3_канал', 'суммарное']

In [309]:
fig = go.Figure(data=[
    go.Bar(name='SGDClassifier', x=list(m_sgd.keys()), y=list(m_sgd.values())),
    go.Bar(name='SVC', x=list(m_svc.keys()), y=list(m_svc.values())),
    go.Bar(name='KNeighbors', x=list(m_kn.keys()), y=list(m_kn.values())),
    go.Bar(name='RandomForest', x=list(m_rf.keys()), y=list(m_rf.values())),
    go.Bar(name='CatBoost', x=list(m_cb.keys()), y=list(m_cb.values())),
])


fig.update_yaxes(title_text="Accuracy", range = (0.6,0.85))
fig.update_xaxes(title_text="Канал (цвет)")
fig.update_layout(title = 'Accuracy моделек обученных на разных каналах + ансамбль')
fig.show()

- хуже всех модельки обучаются на 1 канале (красный цвет)
- лучше всех на голубом
- на SVC и RandomForest суммарная модель показывает лучшие результаты чем на отдельных каналах (проверить на стабильность)
- лучший результат у SVC (0.84)